# Freeman's Word Matching Problem



This problem is defined by James Freemand in his book Simulating Neural Networks with Mathematica.
The problem is studied in Chapter 9, Introduction to Genetic Algorithms by S.N. Sivanandam.

The wordmatching problem tries to evolve an expression of ''to be or not to be'' from the randomly generated lists of letters with genetic algorithm. Since there are 26 possible letters for each of 13 locations in the list, the probability that we get the
correct phrase in a pure random way is (1/26)13 = 4.03038∗10e−19, which is about two chances out of a billion.

We use a list of ASCII integers to encode the string of letters. The lower case
letters in ASCII are represented by numbers in the range [97,122] in the decimal
number system. For example, the string of letters `tobeornottobe` is converted into
the following chromosome represented with ASCII integers:
[116,111,98,101,111,114,110,111,116,116,111,98,101].  Lowercase ASCII character have integer codes

In [Learning DEAP from examples](https://www.amazon.com/Learning-DEAP-examples-Evolutionary-evolutionary-ebook/dp/B06XHXD2SF) Ronn Kling implements the problem in DEAP, solving for a four letter word such as `test` [116,101,115,116].  Keep to lowercase  and don't use spaces.

The DEAP scripts can all be quite similar if the standard DEAP functions are used.
See the [02-travelling-salesman.ipynb](https://github.com/NelisW/NeuralNetworks-DeepLearning-Notes/blob/master/GAGP/DEAP/code/02-travelling-salesman.ipynb) notebook for more detail on the standard DEAP code, the code will be used here with minimal additional comments.




In [1]:
import matplotlib.pyplot as plt
import sys
import array
import random
import numpy as np

from deap import  algorithms
from deap import  base
from deap import  creator
from deap import  tools

%matplotlib inline

In [3]:
# the word to be constructed
string = 'test'
lenstr = len(string)
# convert to ASCII representation
stringToMatch = [ord(c) for c in string]
# create fitness method
creator.create("FitnessMin", base.Fitness,weights=tuple([-1.0 for i in string] ))
# create individual method
creator.create("Individual",list, fitness=creator.FitnessMin)
# toolbox
toolbox = base.Toolbox()
# create the random ASCII attributes: a->97, z->121 (both values inclusive)
toolbox.register('attrACSII',random.randint,97,122)


C:\ProgramData\Anaconda3\lib\site-packages\deap-1.2.2-py3.7-win-amd64.egg\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
